In [ ]:
# Install required libraries
%pip install mediapipe opencv-python

In [1]:
# Import libraries
import cv2
import mediapipe as mp
from threading import Thread
# Initialize MediaPipe hands module
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

In [3]:
import cv2
import mediapipe as mp


# Function to preprocess the frame and detect piano keys
def detect_piano_keys(frame):
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply GaussianBlur to reduce noise and help with edge detection
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Use Canny edge detector to find edges
    edges = cv2.Canny(blurred, 50, 150)

    # Find contours in the edged image
    contours, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Get the height and width of the frame
    height, width, _ = frame.shape

    # Define the region corresponding to the piano keyboard
    piano_keyboard_region = [0, int(height * 0.7), width, height]

    return frame


# Function to process webcam frames
def process_webcam():
    # Open the webcam
    cap = cv2.VideoCapture(0)  # 0 corresponds to the default webcam

    # Initialize MediaPipe hands
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands()

    while cap.isOpened():
        # Read a frame from the webcam
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the frame to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the frame with MediaPipe hands
        results = hands.process(frame_rgb)
        
        # If hands are detected, draw landmarks on the frame
        if results.multi_hand_landmarks:
            for landmarks in results.multi_hand_landmarks:
                # Draw landmarks
                mp.solutions.drawing_utils.draw_landmarks(frame, landmarks, mp_hands.HAND_CONNECTIONS)
                # Get landmarks' coordinates
                for idx, landmark in enumerate(landmarks.landmark):
                    height, width, _ = frame.shape
                    #handedness = landmark.handedness[idx]                      #This line of code is used to find the handedness DEBUGG
                    cx, cy = int(landmark.x * width), int(landmark.y * height)
                    
                    # Draw circles at the tips and proximal phalanges
                    if idx in [mp_hands.HandLandmark.MIDDLE_FINGER_TIP.value,
                               mp_hands.HandLandmark.MIDDLE_FINGER_PIP.value, 
                               mp_hands.HandLandmark.THUMB_TIP.value,           #Obtains values of pinky and thumb
                               mp_hands.HandLandmark.PINKY_TIP.value]:
                        cv2.circle(frame, (cx, cy), 5, (255, 0, 0), cv2.FILLED)

                        # Draw X and Y axes
                        #cv2.line(frame, (0, cy), (width, cy), (255, 0, 0), 2)  # Draw horizontal line (X axis)

                        # Check if the Y-axis of the tip is close to the Y-axis of the PIP
                        y_tip = landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP.value].y
                        y_pip = landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_PIP.value].y

                        x_pinky = landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP.value].x
                        x_thumb = landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP.value].x

                        cv2.putText(frame, "Distance between pinky and thumb is " + str(round(abs(x_pinky - x_thumb),2)), 
                                    (50,50), cv2.FONT_HERSHEY_SIMPLEX,              #Output distance between pinky and thumb for testing
                                        0.5, (255, 0, 0), 2, cv2.LINE_AA)
                        # Set a small range for the check
                        y_range = 0.218
                        x_range = 0.5

                        if (abs(y_tip - y_pip) > y_range) and not(abs(x_pinky - x_thumb) > x_range): #Added condition for pinky and thumb
                            # Display a warning if the hand is flat
                            warning_text = "Hand is flat!"
                            text_size = 2  # Increase text size

                            # Display text at the top right corner
                            cv2.putText(frame, warning_text, (width - 600, 100), cv2.FONT_HERSHEY_SIMPLEX,
                                        text_size, (0, 0, 255), 2, cv2.LINE_AA)
                        
                    #cv2.putText(frame, f"{handedness[0].category_name}" , cv2.FONT_HERSHEY_DUPLEX,
                    #0.75, (0, 0, 255), 2, cv2.LINE_AA)

        # Detect piano keys
        result_frame = detect_piano_keys(frame)

        # Display the frame with landmarks and piano keys
        cv2.imshow('Hand Tracking and Piano Key Detection', result_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the webcam capture object
    cap.release()
    cv2.destroyAllWindows()




In [4]:
# Call the function to process the webcam frames
process_webcam()